In [1]:
import pandas as pd
import re

In [2]:
products = pd.read_csv('../data/credo_product_info.csv')
urls = pd.read_csv('../data/credo_product_slugs.csv')

In [3]:
urls.head()

,category,href
0,all-cleansers,/collections/all-cleansers/products/one-love-o...
1,all-cleansers,/collections/all-cleansers/products/exfoliatin...
2,all-cleansers,/collections/all-cleansers/products/josh-roseb...
3,all-cleansers,/collections/all-cleansers/products/indie-lee-...
4,all-cleansers,/collections/all-cleansers/products/indie-lee-...


In [4]:
products.head()

,name,brand,category,price,ingredients
0,Vitamin B Enzyme Cleansing Oil + Makeup Remover,One Love Organics,all-cleansers,42.0,"helianthus annuus (sunflower) seed oil*, capr..."
1,Exfoliating Instant Facial,Goop,all-cleansers,125.0,"aloe barbadensis (aloe) leaf juice*, vitis vi..."
2,Hydrating Accelerator,Josh Rosebrook,all-cleansers,35.0,"*aloe vera leaf juice, *jojoba oil, †marula s..."
3,Co Q10 Toner,Indie Lee,all-cleansers,34.0,"purified water (aqua), aloe barbadensis, coen..."
4,Brightening Cleanser,Indie Lee,all-cleansers,34.0,"purified water, decyl glucoside (veg. oils & ..."


In [5]:
#adding the urls
#have to first drop the urls that we dropped earlier in product gathering file
urls.drop([50, 77, 161, 204, 253, 273], inplace = True)
urls.reset_index(drop = True, inplace = True)
urls.href = 'https://credobeauty.com/' + urls.href
products['url'] = urls.href

In [6]:
#adding the two products that failed to scrape
products_missing = []
#adding the failed i=273
product = {}
product['name'] = "Le Masque Hydralift Intense"
product['brand'] = "African Botanics"
product['category'] = 'masks-1'
product['price'] = "180.0"
product['url'] = 'https://credobeauty.com/products/le-masque-hydralift-intense'

product['ingredients'] = "aqua (water), squalane, hamamelis virginia (witch hazel) extract, glycerin (botanical), macadamia ternifolia seed oil, phospholipids, glycine soja sterols, stearic acid, safflower oil/palm oil aminopropanediol esters, yeast extract, kappaphycus alvarezii extract, caesalpinia spinosa fruit extract, helichrysum stoechas extract, chlorella sorokiniana extract, saccharide hydrolysate, butylene glycol (sugar cane), sclerocarya birrea (marula)** seed oil, hyaluronic acid, hydrolyzed f-fucoidan (laminaria) algae, hydroxyethyl cellulose, xanthan gum, ascorbyl palmitate (vitamin c), myrothamus flabellifolius (resurrection) plant oil, hydrolyzed candida saitoana extract, niacinamide (vitamin b3), citrillus lanatus (kalahari melon)** seed oil, rosa centifolia (rose)* flower extract, cape aloe (aloe ferox)* leaf, cucumis sativus (cucumber) extract, malva sylvestris (mallow) extract, adansonia digitata (baobab)** seed oil, tetrasodium glutamate diacetate, aspalathus linearis (green rooibos)* tea extract, cyclopia genistoides (honeybush)* leaf extract, sodium hydroxide, ethylhexylglycerin, phenoxyethanol. *organic, **wild-harvested ingredients. "
products_missing.append(product)
#adding the failed i = 611
product = {}
product['name'] = "Watermelon Lip Balm"
product['brand'] = "DedCool"
product['category'] = 'lip-care'
product['price'] = '14.0'
product['url'] = 'https://credobeauty.com/products/watermelon-lip-balm'
product['ingredients'] = "euphorbia cerifera (candelilla) wax, organic cocos nucifera (coconut) oil*, butyrospermum parkii (shea butter), sesamum indicum (sesame seed) oil, aloe barbadensis (aloe vera) oil, borago officinalis (borage oil), macadamia ternifolia (macadamia nut) seed oil, persa gratissima (avocado) oil, vitis vinifera (grape) seed oil, tocopherol (vitamin e), watermelon flavor​"
products_missing.append(product)


product_df = pd.DataFrame(products_missing) 
products = pd.concat([products, product_df], axis = 0, ignore_index = True, sort = False)

In [7]:
products[products['name'] == "Le Masque Hydralift Intense"]

,name,brand,category,price,ingredients,url
606,Le Masque Hydralift Intense,African Botanics,masks-1,180.0,"aqua (water), squalane, hamamelis virginia (wi...",https://credobeauty.com/products/le-masque-hyd...


In [8]:
#dropping duplicate occurance by 'name' and 'brand' of product
products.drop_duplicates(['name', 'brand'], keep="first", inplace = True)

#### Price

In [9]:
#converting the price to float
products.price = products.price.astype(float)

#### Ingredients

In [10]:
products['ingredients'] = products['ingredients'].str.replace('*', "")
products['ingredients'] = products['ingredients'].str.replace('\xa0', " ")
products['ingredients'] = products['ingredients'].str.replace('\n', " ")
products['ingredients'] = products['ingredients'].str.replace(';', " ")
products['ingredients'] = products['ingredients'].str.replace('aroma•', " ")
products['ingredients'] = products['ingredients'].str.replace('†', " ")
products['ingredients'] = products['ingredients'].str.replace('★', " ")
products['ingredients'] = products['ingredients'].str.replace('‡', " ")
products['ingredients'] = products['ingredients'].str.replace('°', " ")
products['ingredients'] = products['ingredients'].str.replace('§', " ")

products['ingredients'] = products['ingredients'].str.replace('Read More Read Less', " ")

In [11]:
for i in products.index:
    #remove anything between parethesis and parenthesis
    new_string = re.sub(r'\([^)]*\)', '', products.loc[i, 'ingredients'])
    products.loc[i, 'ingredients'] = new_string.replace('  ', ' ')
    
#remove everything after /
for i in products.index:
    list_of_ingredients = []
    split = products.ingredients[i].split(", ")
    for j in range(len(split)):
        results = split[j].split("/")[0]
        results = results.strip() #stripping any white space 
        list_of_ingredients.append(results)
    products.loc[i, 'ingredients'] = ', '.join(list_of_ingredients)
    
#remove everything after -
for i in products.index:
    list_of_ingredients = []
    split = products.ingredients[i].split(", ")
    for j in range(len(split)):
        results = split[j].split("-")[0]
        results = results.strip() #stripping any white space 
        list_of_ingredients.append(results)
    products.loc[i, 'ingredients'] = ', '.join(list_of_ingredients)
    
#remove everything after :
for i in products.index:
    list_of_ingredients = []
    split = products.ingredients[i].split(", ")
    for j in range(len(split)):
        results = split[j].split(":")[0]
        results = results.strip() #stripping any white space 
        list_of_ingredients.append(results)
    products.loc[i, 'ingredients'] = ', '.join(list_of_ingredients)
    
    
#remove everything after .
for i in products.index:
    list_of_ingredients = []
    split = products.ingredients[i].split(", ")
    for j in range(len(split)):
        results = split[j].split(".")[0]
        results = results.strip() #stripping any white space 
        list_of_ingredients.append(results)
    products.loc[i, 'ingredients'] = ', '.join(list_of_ingredients)

In [12]:
# #this is a quick method to check the ingredients for legibility

# all_ingredients = []

# for i in products.index:
#     list_ingredients = products.ingredients[i].split(', ')
#     for j in list_ingredients:     
#         all_ingredients.append(j.strip())
# print(len(all_ingredients))
# print(len(set(all_ingredients)))

# unique_ingredients = set(all_ingredients)
# sorted(unique_ingredients)[1000:2000]

In [13]:
products['type'] = 'clean'
products['store'] = "Credo"

In [14]:
products.to_csv("../data/credo_initial_clean.csv", index = False)